In [1]:
import pandas as pd
import re
from nltk.corpus import words

In [2]:
data = pd.read_csv('listings_df.csv')

In [3]:
#drop residual index from csv
data.drop(data.columns.values[0], axis = 1, inplace = True)
data = data[data.notnull()]
data = data.fillna('No Info')

In [4]:
#function to get numbers from string
def get_nums(cell):
    match = re.search('[0-9.,]+', cell)
    if match:
        return match.group()
    else:
        return 'No Info'

In [5]:
num_list = ['rooms', 'beds', 'bath', 'sqft', 'price/ft']
for col in num_list:
    data[col] = data[col].apply(get_nums)

In [6]:
#characters to remove from list of items
remove_list = ['[', ']', "'"]

In [7]:
amenities_list = []

unique_amenities = set()
max_len = 0

In [8]:
for pack in data['amenities'].values.tolist():
    new_pack = pack
    for char in remove_list:
        new_pack = new_pack.replace(char, '')
    unpack = new_pack.split(',')
    unpack = [item.strip() for item in unpack if item.strip() and'google' not in item]
    
    max_len = max(max_len, len(unpack))
    for item in unpack:
        unique_amenities.add(item)

In [9]:
for item in unique_amenities:
    if not 'NYC Storm' in item:
        amenities_list.append(item)

In [10]:
for item in amenities_list:
        data[item] = data.apply(lambda x: item in x['amenities'], axis = 1)

In [11]:
data[amenities_list] = data[amenities_list].astype(int)

In [12]:
data['price'] = data['price'].apply(lambda x: x.replace(',', ''))

In [13]:
for char in remove_list: 
    data['coord'] = data['coord'].str.replace(char, '')

In [14]:
data['coord'] = data['coord'].str.split(',')

In [15]:
data['state_code'] = data['state'].apply(lambda x: x[5:7])

In [16]:
data['latitude'] = data['coord'].apply(lambda x: x[0] if len(x) > 1 else 'No Info')
data['longitude'] = data['coord'].apply(lambda x: x[1] if len(x) > 1 else 'No Info')

In [17]:
mapping_df = pd.read_csv('neighborhoods_grouped.csv')
map_dict = dict(zip(mapping_df['old'], mapping_df['new']))

In [18]:
data['neighborhood'] = data['Neighborhood'].replace(map_dict)

In [19]:
word_set = set()
word_checker = set(words.words())
for desc in data['desc']:
    cleaned = desc
    for char in remove_list: 
        cleaned = cleaned.replace(char, '')
    split_str = cleaned.split()
    for word in split_str:
        if word in word_checker:
            word_set.add(word)

In [20]:
data.columns.values.tolist()

['file_name',
 'title',
 'address',
 'desc',
 'url',
 'coord',
 'state',
 'price',
 'rooms',
 'beds',
 'bath',
 'sqft',
 'price/ft',
 'Neighborhood',
 'details',
 'amenities',
 'subway',
 'days on market',
 'availibility',
 'Valet Parking',
 'Roof Rights',
 'Doorman',
 'Part-time Doorman',
 'Live-in Super',
 'Fireplace: Decorative',
 'Fireplace: Wood',
 'Pied-a-Terre Allowed',
 'Laundry in Building',
 'Terrace',
 'View: City',
 'Guarantors Accepted',
 'Garden',
 'Pets Allowed',
 'Parking Available',
 'Wheelchair Access',
 'Valet',
 'View: Garden',
 'Gym',
 'Waterview',
 'Cold Storage',
 'Cats and Dogs Allowed',
 'Central Air Conditioning',
 'Green Building',
 'Board Approval Required',
 'Cats Only - No Dogs',
 'Smoke-free',
 'Balcony',
 'Concierge',
 'Fireplace',
 'Co-purchase Allowed',
 'Dishwasher',
 '"Childrens Playroom"',
 'Package Room',
 'Swimming Pool',
 'Gifts Allowed',
 'Courtyard',
 'Locker/Cage',
 'Storage Available',
 'Sublet',
 'Parents Buying Allowed',
 'Furnished',
 'Pri

In [21]:
data['days'] = data['days on market'].str.split('\n').apply(lambda x: x[-2].strip() if len(x) > 2 else 'No Info')

In [22]:
data['days'] = data['days'].str.split(' ').apply(lambda x: x[0] if len(x) > 2 else 'No Info')

In [23]:
data.head()

,file_name,title,address,desc,url,coord,state,price,rooms,beds,...,Virtual Doorman,Loft,Washer/Dryer In-Unit,Guarantors Allowed,Garage Parking,state_code,latitude,longitude,neighborhood,days
0,10.html,For Rent: 74 Clarke Ave. in Jersey City,74 Clarke Avenue #22E,['74 CLARKE AVENUE #22E is a rental unit in Je...,['https://streeteasy.com/building/74-clarke-av...,"[, ]",['US-NJ'],1749,2,3,...,0,0,0,0,0,NJ,,,NJ,No Info
1,100.html,For Rent: 1214 5th Ave. in Upper Carnegie Hill,1214 5th Avenue #29B,['1214 5th Ave #29B is a rental unit in Upper ...,['https://streeteasy.com/building/1214-fifth-a...,"[40.79140091, -73.95240021]",['US-NY'],5030,3,1,...,0,0,0,0,1,NY,40.79140091,-73.95240021,Upper East Side,9
2,1000.html,For Rent: 3333 Henry Hudson Parkway in Riverdale,3333 Henry Hudson Parkway #18A,['3333 Henry Hudson Parkway #18A is a rental u...,['https://streeteasy.com/building/3333-henry-h...,"[40.88499832, -73.9151001]",['US-NY'],4195,4,2,...,0,0,0,0,1,NY,40.88499832,-73.9151001,Riverdale,No Info
3,10000.html,For Rent: 44 South Oxford St. in Fort Greene,44 South Oxford Street #2R,['44 SOUTH OXFORD STREET #2R is a rental unit ...,['https://streeteasy.com/building/44-south-oxf...,"[40.68832267, -73.97415708]",['US-NY'],3000,3,1,...,0,0,0,0,0,NY,40.68832267,-73.97415708,Fort Greene,No Info
4,10001.html,For Rent: 25 West 68th St. in Lincoln Square,25 West 68th Street #6D,['25 WEST 68 STREET #6D is a rental unit in Li...,['https://streeteasy.com/building/25-west-68-s...,"[40.77435248, -73.97889675]",['US-NY'],4400,3,1,...,0,0,0,0,0,NY,40.77435248,-73.97889675,Upper West Side,No Info


In [25]:
data.to_csv('cleaned_df.csv')

In [26]:
data['sqft'] = data['sqft'].apply(lambda x: x.replace(',', ''))

In [27]:
import ast

In [29]:
data['subway_list'] = data['subway'].apply(ast.literal_eval)

In [39]:
max_len = 0
for sub_list in data['subway_list']:
    if len(sub_list) > max_len:
        max_len = len(sub_list)

In [45]:
for c in range(1,6):
    data['station_' + str(c)] = 0

In [44]:
for i, l in data['subway_list'].iteritems():
    for ind, sub_dict in enumerate(l):
        data.loc[i, 'station_' + str(ind + 1)] = sub_dict['station']

ValueError: Must have equal len keys and value when setting with an iterable